In [ ]:
import pandas as pd
from urllib import request

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', 'raise')

In [ ]:
needsUpdate = False

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
from time import sleep
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

class DateProvider:
    
    INTENSIVSTATIONEN_DATE_FORMAT = "%d.%m.%Y, %H:%M Uhr"

    def __init__(self):
        self.lastUpdated = None
        self.lastUpdatedDataSource = None

    def needsUpdate(self):
        return self.getLastUpdated() < self.getLastUpdatedDataSource()
        
    def getLastUpdated(self):
        if self.lastUpdated is None:
            htmlContent = requests.get("https://knollfrank.github.io/HowBadIsMyBatch/intensivstationen.html").text
            soup = BeautifulSoup(htmlContent, "lxml")
            dateStr = soup.find(id = "Datenstand").text
            self.lastUpdated = datetime.strptime(dateStr, DateProvider.INTENSIVSTATIONEN_DATE_FORMAT)
        
        return self.lastUpdated

    def getLastUpdatedDataSource(self):
        if self.lastUpdatedDataSource is None:
            html = self._getOriginalHtml()
            lastUpdatedColumn = 'Letzte Änderung'
            dataFrame = self._asDataFrame(html, lastUpdatedColumn)
            self.lastUpdatedDataSource = dataFrame.loc['Landkreis-Daten', lastUpdatedColumn].to_pydatetime()

        return self.lastUpdatedDataSource

    def _getOriginalHtml(self):
        options = Options()
        options.headless = True
        driver = webdriver.Firefox(options = options)
        driver.get('https://www.intensivregister.de/#/aktuelle-lage/downloads')
        sleep(10)
        innerHTML = driver.execute_script("return document.body.innerHTML")
        driver.quit()
        return innerHTML

    def _asDataFrame(self, html, lastUpdatedColumn):
        dataFrame = pd.read_html(html, parse_dates = [lastUpdatedColumn])[0]
        dataFrame[lastUpdatedColumn] = pd.to_datetime(dataFrame[lastUpdatedColumn], format = "%d.%m.%Y %H:%M Uhr")
        dataFrame.set_index('Name', inplace = True)
        return dataFrame


In [ ]:
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate: ', needsUpdate)

In [ ]:
from bs4 import BeautifulSoup

class HtmlTransformerUtil:
    
    def applySoupTransformerToFile(self, file, soupTransformer):
        self._writeSoup(soupTransformer(self._readSoup(file)), file)

    def _readSoup(self, file):
        with open(file) as fp:
            soup = BeautifulSoup(fp, 'lxml')
        return soup

    def _writeSoup(self, soup, file):
        with open(file, "w") as fp:
            fp.write(str(soup))    


In [ ]:
def saveLastUpdatedIntensivstationen(lastUpdated):
    def setLastUpdated(soup):
        soup.find(id = "Datenstand").string.replace_with(lastUpdated.strftime(DateProvider.INTENSIVSTATIONEN_DATE_FORMAT))
        return soup

    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../../docs/intensivstationen.html",
        soupTransformer = setLastUpdated)

In [ ]:
saveLastUpdatedIntensivstationen(dateProvider.getLastUpdatedDataSource())

In [ ]:
def readTimeseries(download = False):
    timeSeriesFile = 'zeitreihe-tagesdaten.csv'
    if download:
        _downloadTimeseries(timeSeriesFile)

    timeseries = pd.read_csv(
        timeSeriesFile,
        low_memory = False,
        usecols = ['date', 'bundesland', 'gemeindeschluessel', 'betten_belegt', 'betten_frei'],
        parse_dates = ['date'],
        date_parser = lambda dateStr: pd.to_datetime(dateStr, format = "%Y-%m-%d"),
        dtype = {
            'gemeindeschluessel': 'string',
            'bundesland': 'string'
            })
    return timeseries.sort_values(by = 'date', ascending = True)

# download https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv or https://www.intensivregister.de/#/aktuelle-lage/downloads
def _downloadTimeseries(timeSeriesFile):
    request.urlretrieve(
        'https://diviexchange.blob.core.windows.net/%24web/zeitreihe-tagesdaten.csv',
        timeSeriesFile)


In [ ]:
timeSeries = readTimeseries(download = needsUpdate)
timeSeries

In [ ]:
def readKreise(download = False):
    kreiseFile = '04-kreise.xlsx'
    if download:
        _downloadKreise(kreiseFile)
    
    kreise = pd.read_excel(
        kreiseFile,
        sheet_name = 'Kreisfreie Städte u. Landkreise',
        header = 5,
        index_col = 0)
    kreise = kreise.rename(columns = {'2': 'Bundesland', 3: 'Kreis', 6: 'Einwohnerzahl'})[['Bundesland', 'Kreis', 'Einwohnerzahl']]
    kreise.index.set_names("Key", inplace = True)
    return kreise

# download https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.xlsx?__blob=publicationFile or https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html
def _downloadKreise(kreiseFile):
    request.urlretrieve(
        'https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.xlsx?__blob=publicationFile',
        kreiseFile)


In [ ]:
kreise = readKreise(download = False)
kreise

In [ ]:
class ColumnsAdder:

    def __init__(self, kreise):
        self.kreise = kreise

    def addKreisAndBundeslandAndEinwohnerzahlColumns(self, dataFrame):
        dataFrame_kreise = pd.merge(dataFrame, self.kreise, how = 'left', left_on = 'gemeindeschluessel', right_index = True)
        dataFrame['Kreis'] = dataFrame_kreise['Kreis']
        dataFrame['Einwohnerzahl'] = dataFrame_kreise['Einwohnerzahl']
        return self._addBundeslandColumn(dataFrame)
        
    def _addBundeslandColumn(self, dataFrame):
        return pd.merge(
            dataFrame,
            self._createBundeslandByKeyTable(),
            how = 'left',
            left_on = 'bundesland',
            right_index = True)

    def _createBundeslandByKeyTable(self):
        return self.kreise[self.kreise.index.str.len() == 2][['Bundesland']]


In [ ]:
timeSeries = ColumnsAdder(kreise).addKreisAndBundeslandAndEinwohnerzahlColumns(timeSeries)
timeSeries

In [ ]:
kreisValues = sorted(timeSeries['Kreis'].drop_duplicates().values)

In [ ]:
def getKreisOptions(kreisValues):
    return [getKreisOption(kreis) for kreis in kreisValues]

def getKreisOption(kreis):
    return '<option value="{kreis}">{kreis}</option>'.format(kreis = kreis)


In [ ]:
kreisOptions = ['<option selected="" value="de">Alle Landkreise</option>']  + getKreisOptions(kreisValues)

In [ ]:
from bs4 import BeautifulSoup


class KreisOptionsSetter:

    def setKreisOptions(self, html, options):
        soup = self._setKreisOptions(self._parse(html), self._parseOptions(options))
        return str(soup)

    def _setKreisOptions(self, soup, options):
        kreisSelect = soup.find(id = "kreisSelect")
        kreisSelect.clear()
        for option in options:
            kreisSelect.append(option)
        return soup

    def _parseOptions(self, options):
        return [self._parse(option).option for option in options]

    def _parse(self, html):
        return BeautifulSoup(html, 'lxml')


In [ ]:
import unittest


class KreisOptionsSetterTest(unittest.TestCase):

    def test_setKreisOptions(self):
        # Given
        kreisOptionsSetter = KreisOptionsSetter()

        # When
        htmlActual = kreisOptionsSetter.setKreisOptions(
            html = '''
            <html>
              <body>
                <p>Test<p/>
                <select id="kreisSelect" name="kreis">
                  <option selected="" value="de">Alle Landkreise</option>
                  <option value="Ahrweiler">Ahrweiler</option>
                  <option value="Wiesbaden, Landeshauptstadt">Wiesbaden, Landeshauptstadt</option>
                  <option value="Aichach-Friedberg">Aichach-Friedberg</option>
                </select>
              </body>
            </html>
            ''',
            options = [
                '<option selected="" value="de">Alle Landkreise</option>',
                '<option value="Ahrweiler">Ahrweiler</option>',
                '<option value="Aichach-Friedberg">Aichach-Friedberg</option>'])

        # Then
        assertEqualHTML(
            htmlActual,
            '''
            <html>
              <body>
                <p>Test<p/>
                <select id="kreisSelect" name="kreis">
                  <option selected="" value="de">Alle Landkreise</option>
                  <option value="Ahrweiler">Ahrweiler</option>
                  <option value="Aichach-Friedberg">Aichach-Friedberg</option>
                </select>
              </body>
            </html>
            ''')

# adapted from https://stackoverflow.com/questions/8006909/pretty-print-assertequal-for-html-strings
def assertEqualHTML(string1, string2, file1='', file2=''):
    u'''
    Compare two unicode strings containing HTML.
    A human friendly diff goes to logging.error() if they
    are not equal, and an exception gets raised.
    '''
    from bs4 import BeautifulSoup as bs
    import difflib

    def short(mystr):
        max = 20
        if len(mystr) > max:
            return mystr[:max]
        return mystr
    p = []
    for mystr, file in [(string1, file1), (string2, file2)]:
        if not isinstance(mystr, str):
            raise Exception(u'string ist not unicode: %r %s' %
                            (short(mystr), file))
        soup = bs(mystr)
        pretty = soup.prettify()
        p.append(pretty)
    if p[0] != p[1]:
        for line in difflib.unified_diff(p[0].splitlines(), p[1].splitlines(), fromfile=file1, tofile=file2):
            display(line)
        display(p[0], ' != ', p[1])
        raise Exception('Not equal %s %s' % (file1, file2))


In [ ]:
unittest.main(argv = [''], verbosity = 2, exit = False)

In [ ]:
from bs4 import BeautifulSoup


def saveKreisOptions(kreisOptions):
    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../../docs/intensivstationen.html",
        soupTransformer =
            lambda soup:
                BeautifulSoup(
                    KreisOptionsSetter().setKreisOptions(html = str(soup), options = kreisOptions),
                    'lxml'))


In [ ]:
saveKreisOptions(kreisOptions)

In [ ]:
import os


class IOUtils:

    def saveDictAsJson(dict, file):
        IOUtils.ensurePath(file)
        with open(file, 'w') as outfile:
            json.dump(dict, outfile)

    @staticmethod
    def ensurePath(file):
        directory = os.path.dirname(file)
        if not os.path.exists(directory):
            os.makedirs(directory)


In [ ]:
def getIntensiveCareBeds(timeSeries, kreis = None):
    if kreis is not None:
        return timeSeries[timeSeries['Kreis'] == kreis][['date', 'betten_belegt', 'betten_frei', 'Einwohnerzahl']]
    else:
        return timeSeries.groupby('date').agg(**{
                        'betten_belegt': pd.NamedAgg(column = 'betten_belegt', aggfunc = 'sum'),
                        'betten_frei':   pd.NamedAgg(column = 'betten_frei',   aggfunc = 'sum'),
                        'Einwohnerzahl': pd.NamedAgg(column = 'Einwohnerzahl', aggfunc = 'sum') 
                    }).reset_index()

In [ ]:
import json


def getAndPersistIntensiveCareBeds(timeSeries, kreis = None):
    intensiveCareBeds = getIntensiveCareBeds(timeSeries, kreis)
    display(kreis)
    _saveAsJson(intensiveCareBeds, _getFilename(kreis))
    return intensiveCareBeds


def _saveAsJson(intensiveCareBeds, file):
    IOUtils.saveDictAsJson(
        {
            'population': int(intensiveCareBeds.iloc[0]['Einwohnerzahl']),
            'data': _intensiveCareBeds2Dict(intensiveCareBeds),
        },
        file)


def _intensiveCareBeds2Dict(intensiveCareBeds):
    df = intensiveCareBeds[['date', 'betten_belegt', 'betten_frei']].copy()
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    return df.to_dict(orient = "records")


def _getFilename(kreis):
    return '../../docs/data/intensivstationen/intensivstationen-{suffix}.json'.format(suffix = _getSuffix(kreis))


def _getSuffix(kreis):
    return kreis if kreis is not None else 'de'


In [ ]:
getAndPersistIntensiveCareBeds(timeSeries)

In [ ]:
for kreis in kreisValues:
    getAndPersistIntensiveCareBeds(timeSeries, kreis)

In [ ]:
def publish():
    %cd /home/frankknoll/Dokumente/Corona/projects/HowBadIsMyBatch-pages
    ! git add -A
    ! git commit -m "updating data"
    ! git push

In [ ]:
publish()